In [4]:
import pandas as pd
import numpy as np
import os
import re

#set data paths
os.chdir("/Users/quinnmackay/Documents/GitHub/BICC/Holcene Revision Work")
links_path = 'EDML LC/Data Files/NGRIP_EDML_match.xlsx'
GICC_GRIP_age = "EDML LC/Data Files/GICC05-GICC21 Conversion.xlsx"
EDML_age = 'EDML LC/Data Files/EDML Layer Count.xlsx'

#import data
links = pd.read_excel(links_path, skiprows=11, names=["GICC05 Age", "EDML (m)", "NGRIP (m)", "Max Link Error"], usecols=[0,1,2,3])
EDML = pd.read_excel(EDML_age, sheet_name=1, skiprows=1, names=["Depth (m)", "count", 'Year b2k', 'MCE'], usecols=[0,1,2,3])
EDML["age (b1950)"] = EDML["Year b2k"] - 50

#links data GICC21
GICC21_transfer = pd.read_excel(GICC_GRIP_age, sheet_name=4, skiprows=8, usecols=[0,2], names=["GICC21 age", "GICC05 age"])
links['GICC21 Age b1950'] = (np.interp(links["GICC05 Age"], GICC21_transfer['GICC05 age'], GICC21_transfer['GICC21 age']))-50

#get count vals
EDML['total 0.5'] = (EDML['count'] == 0.5).cumsum()

EDML

,Depth (m),count,Year b2k,MCE,age (b1950),total 0.5
0,113.113,1.0,1255.5,24.5,1205.5,0
1,113.166,0.5,1256.0,25.0,1206.0,1
2,113.210,1.0,1257.0,25.0,1207.0,1
3,113.292,1.0,1258.0,25.0,1208.0,1
4,113.367,1.0,1259.0,25.0,1209.0,1
...,...,...,...,...,...,...
16404,867.856,1.0,16760.0,924.0,16710.0,1799
16405,867.883,1.0,16761.0,924.0,16711.0,1799
16406,867.926,1.0,16762.0,924.0,16712.0,1799
16407,867.947,1.0,16763.0,924.0,16713.0,1799


In [5]:
EDML_GICC_compare = links[["EDML (m)"]].copy() #create a new dataframe for WDC and GRIP depths using stratigraphic links

#get interp age for depths from layer count
interp_edml_age = np.interp(EDML_GICC_compare["EDML (m)"], EDML["Depth (m)"], EDML["age (b1950)"]) #interpolate depths to get ages
interp_total_0_5 = np.interp(EDML_GICC_compare["EDML (m)"], EDML["Depth (m)"], EDML["total 0.5"]) #interpolate depths to get total 0.5 counts

#add interpolated ages to dataframe
EDML_GICC_compare["EDML age (yr BP1950)"] = interp_edml_age #add interpolated ages to dataframe
interp_total_0_5 = [int(x) for x in interp_total_0_5] #add interpolated total 0.5 counts to dataframe
EDML_GICC_compare["total 0.5"] = interp_total_0_5
EDML_GICC_compare = EDML_GICC_compare.reset_index(drop=True) # reset the index

# add NGRIP depths and ages to dataframe
EDML_GICC_compare["NGRIP (m)"] = links["NGRIP (m)"].copy(deep=True) #add NGRIP depths to dataframe
EDML_GICC_compare["NGRIP age (yr BP1950)"] = links["GICC21 Age b1950"].copy(deep=True) #add NGRIP GICC05 ages to dataframe

# limit to LC and GICC21 range
EDML_GICC_compare = EDML_GICC_compare[EDML_GICC_compare["NGRIP age (yr BP1950)"] > 1250] #limit to LC and GICC21 range
EDML_GICC_compare = EDML_GICC_compare[EDML_GICC_compare["NGRIP age (yr BP1950)"] < 3610] 


In [6]:
#calc diff
EDML_GICC_compare["difference (yr)"] = EDML_GICC_compare["EDML age (yr BP1950)"] - EDML_GICC_compare["NGRIP age (yr BP1950)"] #calculate difference between ages

# this computes the error from that given section, which is the difference between layer counts in GICC and WDC for that section
section_error = np.zeros(len(EDML_GICC_compare)) #create empty list for section error
half_layers_per_section = np.zeros(len(EDML_GICC_compare)) #create empty list for half layers per section
for i in range(0, len(EDML_GICC_compare)):
    if i == 0:
        EDML_error = EDML_GICC_compare["EDML age (yr BP1950)"].iloc[i] - 0 #calculate section error for each row
        GICC_error = EDML_GICC_compare["NGRIP age (yr BP1950)"].iloc[i] - 0 #calculate section error for each row
        half_layers = EDML_GICC_compare["total 0.5"].iloc[i] - 0 #get number of half layers in first section
    else:
        EDML_error = EDML_GICC_compare["EDML age (yr BP1950)"].iloc[i] - EDML_GICC_compare["EDML age (yr BP1950)"].iloc[i-1] #calculate section error for each row
        GICC_error = EDML_GICC_compare["NGRIP age (yr BP1950)"].iloc[i] - EDML_GICC_compare["NGRIP age (yr BP1950)"].iloc[i-1] #calculate section error for each row
        half_layers = EDML_GICC_compare["total 0.5"].iloc[i] - EDML_GICC_compare["total 0.5"].iloc[i-1] #get number of half layers in respective section
    section_error[i] = EDML_error - GICC_error #calculate section error for each row
    half_layers_per_section[i] = half_layers #store number of half layers in respective section

EDML_GICC_compare["section error (EDML-NGRIP) yr"] = section_error #calculate difference between ages
EDML_GICC_compare["half layers in section"] = half_layers_per_section #add number of half layers in respective section to dataframe
EDML_GICC_compare.drop("total 0.5", axis=1, inplace=True)

EDML_GICC_compare

output_path = "EDML LC/EDML-GICC Errors/EDML_GICC_Compare.xlsx"
EDML_GICC_compare.to_excel(output_path, index=False)

print(f"File saved to: {output_path}")

File saved to: EDML LC/EDML-GICC Errors/EDML_GICC_Compare.xlsx
